In [ ]:
import os.path
import sys

import altair as alt
import ipywidgets
from IPython.display import display, HTML
import pandas as pd
import numpy as np

from model.advanced_controls import SOLUTION_CATEGORY
from solution import biogas
from solution import biomass
from solution import concentratedsolar
from solution import improvedcookstoves
from solution import instreamhydro
from solution import insulation
from solution import landfillmethane
from solution import microwind
from solution import offshorewind
from solution import onshorewind
from solution import silvopasture
from solution import smartthermostats
from solution import solarhotwater
from solution import solarpvroof
from solution import solarpvutil
import ui.dd_nb as dd

options = []
options += ['biogas:' + str(scenario) for scenario in biogas.scenarios.keys()]
options += ['biomass:' + str(scenario) for scenario in biomass.scenarios.keys()]
options += ['concentratedsolar:' + str(scenario) for scenario in concentratedsolar.scenarios.keys()]
options += ['improvedcookstoves:' + str(scenario) for scenario in improvedcookstoves.scenarios.keys()]
options += ['instreamhydro:' + str(scenario) for scenario in instreamhydro.scenarios.keys()]
options += ['insulation:' + str(scenario) for scenario in insulation.scenarios.keys()]
options += ['landfillmethane:' + str(scenario) for scenario in landfillmethane.scenarios.keys()]
options += ['microwind:' + str(scenario) for scenario in microwind.scenarios.keys()]
options += ['offshorewind:' + str(scenario) for scenario in offshorewind.scenarios.keys()]
options += ['onshorewind:' + str(scenario) for scenario in onshorewind.scenarios.keys()]
options += ['silvopasture:' + str(scenario) for scenario in silvopasture.scenarios.keys()]
options += ['smartthermostats:' + str(scenario) for scenario in smartthermostats.scenarios.keys()]
options += ['solarhotwater:' + str(scenario) for scenario in solarhotwater.scenarios.keys()]
options += ['solarpvroof:' + str(scenario) for scenario in solarpvroof.scenarios.keys()]
options += ['solarpvutil:' + str(scenario) for scenario in solarpvutil.scenarios.keys()]
solution_choices = ipywidgets.SelectMultiple(
    options=sorted(options, key=dd.scenario_sort_key), description='Scenarios',
    disabled=False, rows=8, layout=ipywidgets.Layout(width='500px')
)
solution_choices

SelectMultiple(description='Scenarios', layout=Layout(width='500px'), options=('biogas:PDS-1p2050-Plausible (B…

In [ ]:
#%%prun -s cumulative -l 200
if len(solution_choices.value):
    solutions = []
    for val in solution_choices.value:
        soln, scenario = val.split(':')
        if soln == 'biogas':
            solutions.append(biogas.Biogas(scenario=scenario))
        elif soln == 'biomass':
            solutions.append(biomass.Biomass(scenario=scenario))
        elif soln == 'concentratedsolar':
            solutions.append(concentratedsolar.ConcentratedSolar(scenario=scenario))
        elif soln == 'improvedcookstoves':
            solutions.append(improvedcookstoves.ImprovedCookStoves(scenario=scenario))
        elif soln == 'instreamhydro':
            solutions.append(instreamhydro.InstreamHydro(scenario=scenario))
        elif soln == 'insulation':
            solutions.append(insulation.Insulation(scenario=scenario))
        elif soln == 'landfillmethane':
            solutions.append(landfillmethane.LandfillMethane(scenario=scenario))
        elif soln == 'microwind':
            solutions.append(microwind.MicroWind(scenario=scenario))
        elif soln == 'offshorewind':
            solutions.append(offshorewind.OffshoreWind(scenario=scenario))
        elif soln == 'onshorewind':
            solutions.append(onshorewind.OnshoreWind(scenario=scenario))
        elif soln == 'silvopasture':
            solutions.append(silvopasture.Silvopasture(scenario=scenario))
        elif soln == 'smartthermostats':
            solutions.append(smartthermostats.SmartThermostats(scenario=scenario))
        elif soln == 'solarhotwater':
            solutions.append(solarhotwater.SolarHotWater(scenario=scenario))
        elif soln == 'solarpvroof':
            solutions.append(solarpvroof.SolarPVRoof(scenario=scenario))
        elif soln == 'solarpvutil':
            solutions.append(solarpvutil.SolarPVUtil(scenario=scenario))
else:
    solutions = [silvopasture.Silvopasture()]

# ------------------ Detailed Results -----------------

def blue_label(text):
    o = ipywidgets.Output()
    div = '<div style="background-color:LightSkyBlue;border:1px solid black;font-weight:bold;text-align:center;font-size:0.875em;">'
    with o:
        display(HTML(div + text + '</div>'))
    return o

def fullname(s):
    return type(s).__name__ + ':' + s.scenario


key_results_label = blue_label('The Key Adoption Results')

# --- Unit Adoption ---

unit_adoption_text = []
rrs_flag, land_flag = False, False
for s in solutions:
    if s.ac.solution_category != SOLUTION_CATEGORY.LAND:
        pds_vs_ref = s.ua.soln_pds_tot_iunits_reqd().loc[2050, 'World'] - s.ua.soln_ref_tot_iunits_reqd().loc[2050, 'World']
        unit_adoption_text.append([fullname(s), pds_vs_ref, s.ua.soln_net_annual_funits_adopted().loc[2050, 'World']])
        rrs_flag = True
    else:
        unit_adoption_text.append([fullname(s), s.ua.soln_net_annual_funits_adopted().loc[2050, 'World']])
        land_flag = True
        
assert not (land_flag and rrs_flag), 'Cannot compare RRS with LAND solutions'
is_rrs = rrs_flag  # convenience variable for RRS/LAND conditionals

if is_rrs:
    unit_adoption_text.append(['', 'TW', 'TWh'])
    unit_adoption_columns = ['Scenario', 'Implementation Unit Adoption Increase in 2050 (PDS vs REF)',
                             'Functional Unit Adoption Increase in 2050 (PDS vs REF)']
else:
    unit_adoption_text.append(['', 'Mha'])
    unit_adoption_columns = ['Scenario', 'Land Unit Adoption Increase in 2050 (PDS vs REF)']

unit_adoption_heading = ipywidgets.Output()
with unit_adoption_heading:
    df = pd.DataFrame(unit_adoption_text, columns=unit_adoption_columns)
    display(HTML(df.style.set_table_styles(dd.dataframe_css_styles).hide_index().render()))
        
# --- Adoption ---

adoption_text = []
for s in solutions:
    if is_rrs:
        adoption_text.append([fullname(s),
                              s.ua.soln_pds_tot_iunits_reqd().loc[2050, 'World'],
                              s.ht.soln_pds_funits_adopted().loc[2050, 'World'],
                              '', '', ''])
    else:
        adoption_text.append([fullname(s), s.ht.soln_pds_funits_adopted().loc[2050, 'World'], '', '', ''])

units = ['', 'TW', 'TWh', '%', '%', '%'] if is_rrs else ['', 'Mha', '%', '%', '%']
adoption_text.append(units)
if is_rrs:
    adoption_columns = ['Scenario', 'Global Solution Implementation Units in 2050',
                        'Global Solution Functional Units in 2050']
else:
    adoption_columns = ['Scenario', 'Global Solution Land Units in 2050']
adoption_columns.extend(['Global Solution Adoption Share - 2014', 'Global Solution Adoption Share - 2020',
                         'Global Solution Adoption Share - 2050'])

adoption_heading = ipywidgets.Output()
with adoption_heading:
    df = pd.DataFrame(adoption_text, columns=adoption_columns)
    display(HTML(df.style.set_table_styles(dd.dataframe_css_styles).hide_index().render()))

if is_rrs:
    # Regional adoption not yet implemented for LAND models due to issues with regional land allocation
    # See: https://docs.google.com/document/d/19sq88J_PXY-y_EnqbSJDl0v9CdJArOdFLatNNUFhjEA/edit
    adoption_iunit_chart = ipywidgets.Output()
    with adoption_iunit_chart:
        df = pd.DataFrame()
        for s in solutions:
            df[fullname(s)] = s.ua.soln_pds_tot_iunits_reqd().loc[2020:2050, 'World']
        iunit_df = df.reset_index().melt('Year', value_name='iunits', var_name='solution')
        iunit_chart = alt.Chart(iunit_df, width=400).mark_line().encode(
            y='iunits',
            x='Year:O',
            color=alt.Color('solution', legend=alt.Legend(orient='top-left')),
            tooltip=['solution', 'iunits', 'Year'],
        ).properties(
            title='World Adoption - Implementation Units (TW)' 
        ).interactive()
        display(iunit_chart)

adoption_funit_chart = ipywidgets.Output()
with adoption_funit_chart:
    df = pd.DataFrame()
    for s in solutions:
        if is_rrs:
            df['Total Market-REF:'+fullname(s)] = s.tm.ref_tam_per_region().loc[2020:2050, 'World']
            df['Total Market-PDS:'+fullname(s)] = s.tm.pds_tam_per_region().loc[2020:2050, 'World']
        else:
            df['Total Land Area:'+fullname(s)] = s.tla_per_region.loc[2020:2050, 'World']
        df['Solution-REF:'+fullname(s)] = s.ht.soln_ref_funits_adopted().loc[2020:2050, 'World']
        df['Solution-PDS:'+fullname(s)] = s.ht.soln_pds_funits_adopted().loc[2020:2050, 'World']
    if is_rrs:
        title, units = 'World Growth of Solution - Functional Units (TWh)', 'funits'
    else:
        title, units = 'World Adoption vs Total Land Area (TLA)', 'land units'
    funit_df = df.reset_index().melt('Year', value_name=units, var_name='solution')
    funit_chart = alt.Chart(funit_df, width=400).mark_line().encode(
            y=units,
            x=alt.X('Year', type='ordinal', scale=alt.Scale(domain=list(range(2015, 2056)))),
            color=alt.Color('solution', legend=alt.Legend(orient='top-right')),
            tooltip=['solution', units, 'Year']
        ).properties(title=title).interactive()
    display(funit_chart)

children = [adoption_iunit_chart, adoption_funit_chart] if is_rrs else [adoption_funit_chart]
adoption_graphs = ipywidgets.HBox(children=children)

# --- Financial ---

financial_results_label = blue_label('The Key Financial Results')

financial_text = []
for s in solutions:
    marginal_first_cost = (s.fc.soln_pds_annual_world_first_cost().loc[:2050].sum() - \
        s.fc.soln_ref_annual_world_first_cost().loc[:2050].sum() - \
        s.fc.conv_ref_annual_world_first_cost().loc[:2050].sum()) / 10**9
    net_operating_savings = ((s.oc.conv_ref_cumulative_operating_cost().loc[2050] - \
                             s.oc.conv_ref_cumulative_operating_cost().loc[2020]) - \
                            (s.oc.soln_pds_cumulative_operating_cost().loc[2050] - \
                             s.oc.soln_pds_cumulative_operating_cost().loc[2020])) / 10**9
    lifetime_operating_savings = s.oc.lifetime_cost_forecast()['Marginal Operating Cost Savings'].sum() / 10**9
    financial_text.append([fullname(s), marginal_first_cost, net_operating_savings, lifetime_operating_savings])
financial_text.append(['', 'Billion USD', 'Billion USD', 'Billion USD'])
financial_columns = ['Scenario', 'Marginal First Cost 2020-2050',
                     'Net Operating Savings 2020-2050', 'Lifetime Operating Savings 2020-2050']

financial_heading = ipywidgets.Output()
with financial_heading:
    df = pd.DataFrame(financial_text, columns=financial_columns)
    display(HTML(df.style.set_table_styles(dd.dataframe_css_styles).hide_index().render()))

cost_graphs = ipywidgets.Output()
with cost_graphs:
    df = pd.DataFrame()
    for s in solutions:
        df['Solution Operating Costs/Savings:'+fullname(s)] = s.oc.soln_pds_annual_operating_cost().loc[2020:2050] / 1000000000
        df['Conventional Operating Costs/Savings:'+fullname(s)] = s.oc.conv_ref_annual_operating_cost().loc[2020:2050] / 1000000000
    cost_df = df.reset_index().melt('Year', value_name='costs', var_name='solution')
    cost_chart = alt.Chart(cost_df, width=350).mark_line().encode(
        y=alt.Y('costs', title='US$B'),
        x=alt.X('Year', type='ordinal', scale=alt.Scale(domain=list(range(2015, 2056)))),
        color=alt.Color('solution', legend=alt.Legend(orient='top-left')),
        tooltip=['solution', 'costs', 'Year'],
    ).properties(
        title='World Operating Cost Difference'
    ).interactive()
    display(cost_chart)


# --- Climate ---

climate_results_label = blue_label('The Key Climate Results')

climate_text = []

if is_rrs:
    for s in solutions:
        climate_text.append([fullname(s), (s.c2.co2eq_mmt_reduced().loc[:, 'World'].sum() / 1000)])
    climate_columns = ['Scenario', 'Total Emissions Reduction']
else:
    for s in solutions:
        # Note the sequestration table in co2calcs doesn't set values to zero outside the study years
        # In the xls there is a separate table which does this. Here, we select the years directly.
        # The xls implemntation also excludes the year 2020, almost certainly by mistake. Thus, this 
        # value will differ slightly from the one given in the Detailed Results xls sheet.
        # See: https://docs.google.com/document/d/19sq88J_PXY-y_EnqbSJDl0v9CdJArOdFLatNNUFhjEA/edit
        climate_text.append([fullname(s), (s.c2.co2_sequestered_global().loc[2020:2050, 'All'].sum() / 1000)])
    climate_columns = ['Scenario', 'Total Atmospheric CO2-eq Reduction']

climate_text.append(['', 'Gt CO2 (2020-2050)'])
climate_heading = ipywidgets.Output()
with climate_heading:
    df = pd.DataFrame(climate_text, columns=climate_columns)
    display(HTML(df.style.set_table_styles(dd.dataframe_css_styles).hide_index().render()))

reduction_graph = ipywidgets.Output()
with reduction_graph:
    df = pd.DataFrame()
    for s in solutions:
        if is_rrs:
            df[fullname(s)] = s.c2.co2eq_mmt_reduced().loc[2020:2050, 'World'].cumsum().mul(0.001)
        else:
            df[fullname(s)] = s.c2.co2_sequestered_global().loc[2020:2050, 'All'].cumsum().mul(0.001)
    if is_rrs:
        title, value_name = 'World Cumulative Emissions Reductions (Gigatons CO2-eq)', 'reduction'
    else:
        title, value_name = 'World Cumulative Carbon Sequestration (Gigatons CO2-eq)', 'sequestration'
    reduction_df = df.reset_index().melt('Year', value_name=value_name, var_name='solution')
    reduction_chart = alt.Chart(reduction_df, width=300).mark_line().encode(
        y=value_name,
        x=alt.X('Year', type='ordinal'),
        color=alt.Color('solution', legend=alt.Legend(orient='top-left')),
        tooltip=['solution', value_name, 'Year'],
    ).properties(
        title=title
    ).interactive()
    display(reduction_chart)

concentration_graph = ipywidgets.Output()
with concentration_graph:
    df = pd.DataFrame()
    for s in solutions:
        df[fullname(s)] = s.c2.co2eq_ppm_calculator().loc[2020:2050, 'CO2-eq PPM']
    concentration_df = df.reset_index().melt('Year', value_name='concentration', var_name='solution')
    concentration_chart = alt.Chart(concentration_df, width=300).mark_line().encode(
        y='concentration',
        x=alt.X('Year', type='ordinal'),
        color=alt.Color('solution', legend=alt.Legend(orient='top-left')),
        tooltip=['solution', 'concentration', 'Year'],
    ).properties(
        title='World Cumulative GHG Concentrations Reduction (Gigatons CO2-eq)'
    ).interactive()
    display(concentration_chart)

emissions_graphs = ipywidgets.HBox(children=[reduction_graph, concentration_graph])

detailed_results = ipywidgets.Output()
with detailed_results:
    display(ipywidgets.VBox([key_results_label, unit_adoption_heading, adoption_heading, adoption_graphs,
                             financial_results_label, financial_heading, cost_graphs,
                             climate_results_label, climate_heading, emissions_graphs]))

# --------------- Variable Meta Analysis --------------

variable_meta_analysis = ipywidgets.Output()
with variable_meta_analysis:
    children = []
    for s in solutions:
        for (name, v) in s.VMAs:
            vma_widget = ipywidgets.Output()
            with vma_widget:
                display(HTML(v.source_data.drop(['Link', 'Source Validation Code', 'License Code'], axis=1).fillna('').to_html()))
            summary_table = ipywidgets.Output()
            with summary_table:
                (mean, high, low) = v.avg_high_low()
                df = pd.DataFrame([[mean, high, low]], columns=['Mean', 'High', 'Low'])
                display(HTML(df.style.format("{:.01f}").set_properties(**{'width': '10em'}).set_table_styles(dd.dataframe_css_styles).hide_index().render()))

            accordion = ipywidgets.Accordion(children=[vma_widget], layout=ipywidgets.Layout(width='90%'))
            accordion.set_title(0, name)
            vma_widget = ipywidgets.Output()
            with vma_widget:
                display(ipywidgets.HBox(children=[summary_table, accordion]))
            children.append(vma_widget)
    display(ipywidgets.VBox(children=children))

# ------------------ First Cost -----------------

children = []
for s in solutions:
    df = pd.concat([s.fc.soln_pds_install_cost_per_iunit().loc[2020:2050],
                    s.fc.soln_ref_install_cost_per_iunit().loc[2020:2050],
                    s.fc.conv_ref_install_cost_per_iunit().loc[2020:2050]],
                   axis=1)
    df.columns = ['SOLN-PDS', 'SOLN-REF', 'CONV-REF']
    df[['SOLN-PDS', 'SOLN-REF', 'CONV-REF']] /= s.fc.fc_convert_iunit_factor
    fc_table = ipywidgets.Output()
    with fc_table:
        display(HTML(df.style.format({'SOLN-PDS':'{:.02f}', 'SOLN-REF':'{:.02f}', 'CONV-REF':'{:.02f}'}).set_table_styles(dd.dataframe_css_styles).render()))
    fc_chart = ipywidgets.Output()
    with fc_chart:
        melted_df = df.reset_index().melt('Year', value_name='cost', var_name='column')
        melted_chart = alt.Chart(melted_df, width=300).mark_line().encode(
            y='cost',
            x='Year:O',
            color=alt.Color('column'),
            tooltip=['column', 'cost', 'Year'],
        ).properties(
            title='First Cost (US$M)'
        ).interactive()
        display(melted_chart)
    children.append(ipywidgets.HBox([fc_table, fc_chart]))

first_cost = ipywidgets.Accordion(children=children)
for i, s in enumerate(solutions):
    first_cost.set_title(i, fullname(s))

# ------------------ Operating Cost -----------------

children = []
for s in solutions:
    df = pd.concat([s.oc.soln_pds_annual_operating_cost().loc[2020:2050],
                    s.oc.conv_ref_annual_operating_cost().loc[2020:2050],
                    s.oc.marginal_annual_operating_cost().loc[2020:2050]],
                   axis=1)
    df.columns = ['PDS', 'REF', 'Marginal']
    df[['PDS', 'REF', 'Marginal']] /= s.oc.conversion_factor
    oc_table = ipywidgets.Output()
    with oc_table:
        display(HTML(df.style.format({'PDS':'{:.02f}', 'REF':'{:.02f}', 'Marginal':'{:.02f}'}).set_table_styles(dd.dataframe_css_styles).render()))
    oc_chart = ipywidgets.Output()
    with oc_chart:
        melted_df = df.reset_index().melt('Year', value_name='cost', var_name='column')
        melted_chart = alt.Chart(melted_df, width=300).mark_line().encode(
            y='cost',
            x='Year:O',
            color=alt.Color('column'),
            tooltip=['column', 'cost', 'Year'],
        ).properties(
            title='Operating Costs (US$M)'
        ).interactive()
        display(melted_chart)
    children.append(ipywidgets.HBox([oc_table, oc_chart]))

operating_cost = ipywidgets.Accordion(children=children)
for i, s in enumerate(solutions):
    operating_cost.set_title(i, fullname(s))

# ------------------ Adoption Data -----------------

children = []
for s in solutions:
    df = pd.concat([s.ht.soln_pds_funits_adopted().loc[2020:2050, 'World'],
                    s.ht.soln_ref_funits_adopted().loc[2020:2050, 'World']],
                   axis=1)
    df.columns = ['PDS', 'REF']
    ad_table = ipywidgets.Output()
    with ad_table:
        display(HTML(df.style.format({'PDS':'{:.02f}', 'REF':'{:.02f}'}).set_table_styles(dd.dataframe_css_styles).render()))
    ad_chart = ipywidgets.Output()
    with ad_chart:
        melted_df = df.reset_index().melt('Year', value_name='adoption', var_name='column')
        melted_chart = alt.Chart(melted_df, width=300).mark_line().encode(
            y='adoption',
            x='Year:O',
            color=alt.Color('column'),
            tooltip=['column', 'adoption', 'Year'],
        ).properties(
            title='Adoption (IUnits)' if is_rrs else 'Adoption (Mha)'
        ).interactive()
        display(melted_chart)
    
    if is_rrs:
        geo_source = alt.topo_feature('data/world_topo_sans_antartica_with_dd_regions.json', 'regions')
    
        ad_abs_geo = ipywidgets.Output()
        with ad_abs_geo:
            pds_per_region = s.ht.soln_pds_funits_adopted().loc[[2050]]
            pds_per_region_melted = pds_per_region.reset_index().melt('Year', value_name='adoption', var_name='region')[['region', 'adoption']]
            geo_abs_chart = alt.Chart(geo_source).mark_geoshape(
                fill='#dddddd',
                stroke='black',
            ).encode(
                color=alt.Color('adoption:Q', scale=alt.Scale(scheme='greens')),
                tooltip=['id:O', alt.Tooltip('adoption:Q', format='.2f')],
            ).transform_lookup(
                lookup='id',
                from_=alt.LookupData(data=pds_per_region_melted, key='region', fields=['adoption'])
            # available projections: https://github.com/d3/d3-3.x-api-reference/blob/master/Geo-Projections.md
            ).project('conicEquidistant').properties(
                width=400,
                height=300,
                title='Regional Adoption (total funits in 2050)'
            )
            display(geo_abs_chart)
        
        ad_pct_geo = ipywidgets.Output()
        with ad_pct_geo:
            pds_percent_per_region = (s.ht.soln_pds_funits_adopted().loc[[2050]] / s.tm.pds_tam_per_region().loc[[2050]]) * 100.0
            pds_percent_per_region_melted = pds_percent_per_region.reset_index().melt('Year', value_name='percent', var_name='region')[['region', 'percent']]
            geo_pct_chart = alt.Chart(geo_source).mark_geoshape(
                fill='#dddddd',
                stroke='black',
            ).encode(
                color=alt.Color('percent:Q', scale=alt.Scale(scheme='greens')),
                tooltip=['id:O', alt.Tooltip('percent:Q', format='.2f')],
            ).transform_lookup(
                lookup='id',
                from_=alt.LookupData(data=pds_percent_per_region_melted, key='region', fields=['percent'])
            # available projections: https://github.com/d3/d3-3.x-api-reference/blob/master/Geo-Projections.md
            ).project('conicEquidistant').properties(
                width=400,
                height=300,
                title='Regional Adoption (% of TAM in 2050)'
            )
            display(geo_pct_chart)

        children.append(ipywidgets.HBox([ad_table, ipywidgets.VBox([ad_chart, ad_abs_geo, ad_pct_geo])]))
    else:
        children.append(ipywidgets.HBox([ad_table, ipywidgets.VBox([ad_chart])]))

adoption_data = ipywidgets.Accordion(children=children)
for i, s in enumerate(solutions):
    adoption_data.set_title(i, fullname(s))

# ------------------   TAM Data   ------------------

children = []
titles = []
for s in solutions:
    if not is_rrs:
        continue
    tm_table_pds = ipywidgets.Output()
    with tm_table_pds:
        df = s.tm.pds_tam_per_region()
        header = '<div style="font-size:16px;font-weight:bold;color:#4d4d4d;background-color:#f7f7f9;width:100%">'
        header += 'PDS TAM per region</div>'
        display(HTML(header + df.style.format('{:.02f}').set_table_styles(dd.dataframe_css_styles).render()))
    tm_table_ref = ipywidgets.Output()
    with tm_table_ref:
        df = s.tm.ref_tam_per_region()
        header = '<div style="font-size:16px;font-weight:bold;color:#4d4d4d;background-color:#f7f7f9;width:100%">'
        header += 'REF TAM per region</div>'
        display(HTML(header + df.style.format('{:.02f}').set_table_styles(dd.dataframe_css_styles).render()))
    tm_geo_pds = ipywidgets.Output()
    with tm_geo_pds:
        pds_tam_per_region = s.tm.pds_tam_per_region().loc[[2050]]
        pds_tam_per_region_melted = pds_tam_per_region.reset_index().melt('Year', value_name='adoption', var_name='region')[['region', 'adoption']]
        source = alt.topo_feature('data/world_topo_sans_antartica_with_dd_regions.json', 'regions')
        geo_chart = alt.Chart(source).mark_geoshape(
            fill='#dddddd',
            stroke='black',
        ).encode(
            color=alt.Color('adoption:Q', scale=alt.Scale(scheme='greens'), legend=None),
            tooltip=['id:O', alt.Tooltip('adoption:Q', format='.2f', title='TAM')],
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(data=pds_tam_per_region_melted, key='region', fields=['adoption'])
        # available projections: https://github.com/d3/d3-3.x-api-reference/blob/master/Geo-Projections.md
        ).project('conicEquidistant').properties(
            width=250,
            height=180,
            title='Regional PDS TAM in 2050'
        )
        display(geo_chart)
    tm_geo_ref = ipywidgets.Output()
    with tm_geo_ref:
        ref_tam_per_region = s.tm.ref_tam_per_region().loc[[2050]]
        ref_tam_per_region_melted = ref_tam_per_region.reset_index().melt('Year', value_name='adoption', var_name='region')[['region', 'adoption']]
        source = alt.topo_feature('data/world_topo_sans_antartica_with_dd_regions.json', 'regions')
        geo_chart = alt.Chart(source).mark_geoshape(
            fill='#dddddd',
            stroke='black',
        ).encode(
            color=alt.Color('adoption:Q', scale=alt.Scale(scheme='greens'), legend=None),
            tooltip=['id:O', alt.Tooltip('adoption:Q', format='.2f', title='TAM')],
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(data=ref_tam_per_region_melted, key='region', fields=['adoption'])
        # available projections: https://github.com/d3/d3-3.x-api-reference/blob/master/Geo-Projections.md
        ).project('conicEquidistant').properties(
            width=250,
            height=180,
            title='Regional REF TAM in 2050'
        )
        display(geo_chart)
    children.append(ipywidgets.VBox([ipywidgets.HBox([tm_table_pds, tm_geo_pds]), ipywidgets.HBox([tm_table_ref, tm_geo_ref])]))
    titles.append(fullname(s))

if is_rrs:
    tam_data = ipywidgets.Accordion(children=children)
    for idx, title in enumerate(titles):
        tam_data.set_title(idx, title)
else:
    tam_data = None

# ------------------ CO2 -----------------

children = []
for s in solutions:
    if is_rrs:
        df = pd.concat([s.c2.co2_mmt_reduced().loc[2020:2050, 'World'],
                        s.c2.co2eq_mmt_reduced().loc[2020:2050, 'World']],
                       axis=1)
        df.columns = ['CO2', 'CO2eq']
    else:
        df = pd.DataFrame((s.c2.co2_sequestered_global().loc[2020:2050, 'All']))
        df.columns = ['CO2']
    c2_table = ipywidgets.Output()
    with c2_table:
        display(HTML(df.style.format('{:.02f}').set_table_styles(dd.dataframe_css_styles).render()))
    c2_chart = ipywidgets.Output()
    with c2_chart:
        melted_df = df.reset_index().melt('Year', value_name='mmt', var_name='column')
        melted_chart = alt.Chart(melted_df, width=300).mark_line().encode(
            y='mmt',
            x='Year:O',
            color=alt.Color('column'),
            tooltip=['column', 'mmt', 'Year'],
        ).properties(
            title='CO2 Reduction (MMT)' if is_rrs else 'CO2 Sequestration (MMT)' 
        ).interactive()
        display(melted_chart)
    children.append(ipywidgets.HBox([c2_table, c2_chart]))

co2_calcs = ipywidgets.Accordion(children=children)
for i, s in enumerate(solutions):
    co2_calcs.set_title(i, fullname(s))
    
# ------------------ AEZ Data -----------------

if not is_rrs:
    children = []
    for s in solutions:
        df = s.ae.get_land_distribution()
        ae_table = ipywidgets.Output()
        with ae_table:
            display(HTML(df.style.format('{:.02f}').set_table_styles(dd.dataframe_css_styles).render()))
        children.append(ipywidgets.HBox([ae_table]))
    
    aez_data = ipywidgets.Accordion(children=children)
    for i, s in enumerate(solutions):
        aez_data.set_title(i, fullname(s))
else:
    aez_data = None


In [ ]:
all_solutions = pd.read_csv(os.path.join('data', 'overview', 'solutions.csv'),
                                    index_col=False, skipinitialspace=True, header=0,
                                    skip_blank_lines=True, comment='#')
soln_results = pd.read_csv(os.path.join('data', 'overview', 'soln_results.csv'),
                                    index_col=False, skipinitialspace=True, header=0,
                                    skip_blank_lines=True, comment='#')
all_solutions = all_solutions.merge(soln_results, on='Solution', how='left')
sectors = all_solutions.pivot_table(index='Sector',aggfunc=sum)
all_solutions['SectorCO2eq'] = all_solutions.apply(lambda row: sectors.loc[row['Sector'], 'CO2eq'], axis=1)

solution_chart = ipywidgets.Output()
with solution_chart:
    display({'application/vnd.vega.v4+json': dd.solution_donut_chart(solutions=all_solutions, width=400, height=400)}, raw=True)

solution_list = ipywidgets.Output()
with solution_list:
    display(HTML((all_solutions.sort_values(['SectorCO2eq', 'CO2eq'], ascending=False)
                  .loc[:,['Solution', 'Sector', 'CO2eq', 'DirName']].fillna('')
                  .style.format({'CO2eq':'{:.02f}'}).set_table_styles(dd.dataframe_css_styles)
                  .hide_index().render())))

solution_treemap = ipywidgets.Output()
with solution_treemap:
    display({'application/vnd.vega.v4+json': dd.solution_treemap(solutions=all_solutions, width=400, height=800)}, raw=True)

charts = ipywidgets.VBox(children=[solution_chart, solution_treemap])
overview = ipywidgets.HBox(children=[solution_list, charts])

In [ ]:
# ------------------ Create tabs -----------------
children = [overview, detailed_results, variable_meta_analysis, adoption_data]
titles = ["Overview", "Detailed Results", "Variable Meta-Analysis", "Adoption Data"]
if tam_data:
    children.append(tam_data)
    titles.append("TAM Data")
children.extend([first_cost, operating_cost, co2_calcs])
titles.extend(["First Cost", "Operating Cost", "CO2"])
if aez_data:
    children.append(aez_data)
    titles.append('AEZ Data')

tabs = ipywidgets.Tab(children=children)
for (idx, title) in enumerate(titles):
    tabs.set_title(idx, title)

display(tabs)